<a href="https://colab.research.google.com/github/fhfmendes04/HA_FJSSP/blob/main/HA_AG_TS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import random


In [2]:
class individuo():
  def __init__(self, job, operation, machine, regras_MS):
    self.OS = []
    self.MS = []
    self.fitness = 0
    self.regras_MS = regras_MS

    self.job = job
    self.operation = operation
    self.machine = machine

    self.OS_String()
    self.MS_String()

  def OS_String(self):
    lista = [3,3,3,3,3,3,3,3,3,3,3,3]
    for l in range(0,self.job * self.operation):
      flag = True
      while flag:
        numero = random.randint(0,11)
        if lista[numero]>=0:
          lista[numero] -= 1
          self.OS.append(numero)
          flag=False

  def MS_String(self):
    for j in range(0, self.job):
      reg = self.regras_MS[j]
      for o in range(0, self.operation):
        self.MS.append(random.choice(reg[o]))


In [193]:
from pandas.core.window.common import find_stack_level
class HA_AG_TS():
  def __init__(self, tamanho_populacao=400, geracao=200, prob_crossover=0.8, prob_mutacao=0.1):
    self.tamanho_populacao = tamanho_populacao
    self.populacao = []
    self.geracao = geracao
    self.prob_crossover = prob_crossover
    self.prob_mutacao = prob_mutacao
    self.lista_solucoes = []
    self.df = pd.DataFrame()
    self.job = 0
    self.operation = 0
    self.machine = 0
    self.regras_MS = []

    self.upload()
    self.regramento_MS()

  def upload(self):
      arquivo = "/content/drive/MyDrive/USP/Mestrado Sistema de Informação/SIN 5006 - Inteligência Computacional/Apresentação I/code/MFJS01.xlsx"
      self.df = pd.read_excel(arquivo, index_col=False)
      self.job = len(self.df['JOB'].unique())
      self.operation = len(self.df['OPERATION'].unique())
      self.machine = len(self.df['MACHINE'].unique())
      print("JOB: ", self.job)
      print("OPERATION: ", self.operation)
      print("MACHINE: ", self.machine)

      self.df.fillna(0,inplace=True)
      self.df["MACHINE.2"] = self.df["MACHINE.2"].astype(int)
      self.df['TIME.2'] = self.df["TIME.2"].astype(int)

  def regramento_MS(self):
    for j in range(0,self.job):
      regra_o = []
      dados = self.df[self.df["JOB"]==j].reset_index(drop=True)
      for o in range(0, self.operation):
        regra = []
        machine_1 = dados.iloc[o][3]
        machine_2 = dados.iloc[o][5]
        machine_3 = dados.iloc[o][7]

        if (machine_1 != 0):
          regra.append(machine_1)
        elif (machine_1 == 0):
          if (dados.iloc[0][4]!= 0):
            regra.append(machine_1)

        if(machine_2 != 0):
          regra.append(machine_2)
        elif (machine_2 == 0):
          if(dados.ilo[o][6] != 0):
            regra.append(machine_2)

        if(machine_3 !=0):
          regra.append(machine_3)
        elif (machine_3 == 0):
          if (dados.iloc[o][8] !=0):
            regra.append(machine_3)

        regra_o.append(regra)

      self.regras_MS.append(regra_o)

  def fitness(self,os_lista, ms_lista):
    maquinas = []
    controle_os = np.zeros(self.job)

    #Transforma a lista de 48 elementos em 12 x 4
    conta = 0
    mostra = []
    maquinas = []
    machine_job = [[0],[0],[0],[0],[0],[0],[0],[0]]
    job_time = [[0],[0],[0],[0],[0],[0],[0],[0],[0],[0],[0],[0]]
    for i in ms_lista:
      mostra.append(i)
      conta +=1
      if ((conta % 4) == 0):
        conta = 0
        maquinas.append(mostra)
        mostra=[]
    #print(maquinas)
    for o in os_lista:
      posicao = int(controle_os[o])
      controle_os[o] += 1
      #print(o,posicao, controle_os, maquinas[o][posicao])
      dados = self.df[self.df["JOB"]==o].reset_index(drop=True)
      localizador = maquinas[o][posicao]
      machine_1 = dados.iloc[posicao][3]
      machine_2 = dados.iloc[posicao][5]
      machine_3 = dados.iloc[posicao][7]

      if (machine_1 == localizador):
        if (dados.iloc[posicao][4]!= 0):
          if (posicao == 0): # Primeira vez na machine
            machine_job[localizador].append("O(" + str(o) + str(posicao) + "):" + str(machine_job[localizador][0]) + "-" + str(machine_job[localizador][0] + dados.iloc[posicao][4]))
            machine_job[localizador][0] = machine_job[localizador][0] + dados.iloc[posicao][4]
            job_time[o] = machine_job[localizador][0]
          else:
              inicio = job_time[o]
              final = job_time[o] + dados.iloc[posicao][4]
              ultimo = final
              for m in range(1, len(machine_job[localizador])):
                  s = machine_job[localizador][m]
                  index_1 = s.find(":")
                  index_2 = s.find("-")
                  numero_1 = int(s[(index_1+1):index_2])
                  numero_2 = int(s[(index_2+1):])
                  if (((inicio > numero_1) and (inicio >= numero_2)) and ((final > numero_1) and (final > numero_2))):
                      continue
                  else:
                    if ((inicio < numero_1) and (final < numero_1) and (ultimo >= inicio)):
                        break
                    else:
                        if ((inicio >= numero_1) and (inicio >= numero_2)):
                          break
                        else:
                          inicio = numero_2
                          ultimo = final
                          final = inicio + dados.iloc[posicao][4]

              machine_job[localizador].append("O(" + str(o) + str(posicao) + "):" + str(inicio) + "-" + str(final))
              machine_job[localizador][0] = final
              job_time[o] = machine_job[localizador][0]
      else:
        if (machine_2 == localizador):
          if (dados.iloc[posicao][4]!= 0):
            if (posicao == 0): # Primeira vez na machine
              machine_job[localizador].append("O(" + str(o) + str(posicao) + "):" + str(machine_job[localizador][0]) + "-" + str(machine_job[localizador][0] + dados.iloc[posicao][4]))
              machine_job[localizador][0] = machine_job[localizador][0] + dados.iloc[posicao][4]
              job_time[o] = machine_job[localizador][0]
            else:
              inicio = job_time[o]
              final = job_time[o] + dados.iloc[posicao][4]
              for m in range(1, len(machine_job[localizador])):
                  s = machine_job[localizador][m]
                  index_1 = s.find(":")
                  index_2 = s.find("-")
                  numero_1 = int(s[(index_1+1):index_2])
                  numero_2 = int(s[(index_2+1):])
                  if (((inicio > numero_1) and (inicio >= numero_2)) and ((final > numero_1) and (final > numero_2))):
                      continue
                  else:
                    if ((inicio < numero_1) and (final < numero_1) and (ultimo >= inicio)):
                        break
                    else:
                        if ((inicio >= numero_1) and (inicio >= numero_2)):
                          break
                        else:
                          inicio = numero_2
                          ultimo = final
                          final = inicio + dados.iloc[posicao][4]

              machine_job[localizador].append("O(" + str(o) + str(posicao) + "):" + str(inicio) + "-" + str(final))
              machine_job[localizador][0] = final
              job_time[o] = machine_job[localizador][0]
        else:
          if (machine_3 == localizador):
            if (dados.iloc[posicao][4]!= 0):
              if (posicao == 0): # Primeira vez na machine
                machine_job[localizador].append("O(" + str(o) + str(posicao) + "):" + str(machine_job[localizador][0]) + "-" + str(machine_job[localizador][0] + dados.iloc[posicao][4]))
                machine_job[localizador][0] = machine_job[localizador][0] + dados.iloc[posicao][4]
                job_time[o] = machine_job[localizador][0]
              else:
                inicio = job_time[o]
                final = job_time[o] + dados.iloc[posicao][4]
                for m in range(1, len(machine_job[localizador])):
                    s = machine_job[localizador][m]
                    index_1 = s.find(":")
                    index_2 = s.find("-")
                    numero_1 = int(s[(index_1+1):index_2])
                    numero_2 = int(s[(index_2+1):])
                    if (((inicio > numero_1) and (inicio >= numero_2)) and ((final > numero_1) and (final > numero_2))):
                      continue
                    else:
                      if ((inicio < numero_1) and (final < numero_1) and (ultimo >= inicio)):
                        break
                      else:
                        if ((inicio >= numero_1) and (inicio >= numero_2)):
                          break
                        else:
                          inicio = numero_2
                          ultimo = final
                          final = inicio + dados.iloc[posicao][4]

              machine_job[localizador].append("O(" + str(o) + str(posicao) + "):" + str(inicio) + "-" + str(final))
              machine_job[localizador][0] = final
              job_time[o] = machine_job[localizador][0]
    for x in machine_job:
      print(x)

    #print(job_time)




In [194]:
algoritmo = HA_AG_TS()
#ind = individuo(12, 4, 8, algoritmo.regras_MS)

JOB:  12
OPERATION:  4
MACHINE:  8


In [126]:
print (ind.OS)

[6, 5, 0, 7, 5, 3, 8, 10, 2, 6, 2, 2, 3, 5, 3, 5, 10, 6, 7, 1, 1, 2, 0, 6, 8, 7, 3, 11, 7, 10, 0, 8, 11, 4, 1, 0, 11, 10, 8, 9, 4, 11, 9, 1, 9, 4, 4, 9]


In [127]:
print(ind.MS)

[0, 1, 6, 7, 2, 2, 6, 4, 1, 3, 5, 5, 0, 2, 5, 4, 0, 4, 4, 5, 1, 3, 6, 3, 2, 2, 5, 4, 3, 5, 4, 7, 1, 2, 6, 7, 3, 4, 5, 6, 2, 2, 6, 5, 2, 2, 4, 4]


In [195]:
algoritmo.fitness(ind.OS, ind.MS)

[1899, 'O(00):0-247', 'O(30):247-334', 'O(40):334-457', 'O(40):1675-1899']
[522, 'O(50):0-145', 'O(80):145-295', 'O(20):295-382', 'O(01):382-522']
[1899, 'O(60):0-145', 'O(100):145-295', 'O(61):295-419', 'O(31):419-669', 'O(101):669-849', 'O(10):849-1063', 'O(11):1063-1150', 'O(81):1150-1330', 'O(110):1330-1675', 'O(111):1675-1899']
[905, 'O(70):0-245', 'O(51):245-395', 'O(21):395-540', 'O(53):540-660', 'O(90):660-905']
[2814, 'O(63):1020-1250', 'O(72):469-614', 'O(33):1250-1500', 'O(112):1899-2044', 'O(41):2044-2130', 'O(113):2130-2360', 'O(91):2360-2584', 'O(13):2584-2704', 'O(42):2704-2814']
[2979, 'O(22):540-730', 'O(32):730-875', 'O(62):875-1020', 'O(71):245-469', 'O(23):1020-1190', 'O(103):1190-1340', 'O(92):2584-2729', 'O(43):2814-2979']
[2879, 'O(52):395-515', 'O(102):849-889', 'O(02):522-672', 'O(82):1330-1370', 'O(12):1150-1328', 'O(93):2729-2879']
[1520, 'O(73):614-764', 'O(03):764-974', 'O(83):1370-1520']


In [ ]:
ind.OS

[4,
 9,
 10,
 7,
 0,
 4,
 3,
 0,
 4,
 9,
 11,
 10,
 10,
 9,
 4,
 11,
 2,
 0,
 6,
 10,
 0,
 1,
 1,
 11,
 1,
 6,
 8,
 3,
 8,
 11,
 9,
 6,
 3,
 6,
 7,
 5,
 2,
 2,
 8,
 5,
 1,
 2,
 8,
 5,
 5,
 3,
 7,
 7]

In [ ]:
ind.MS

[1,
 3,
 6,
 7,
 0,
 2,
 4,
 4,
 1,
 3,
 5,
 7,
 1,
 2,
 5,
 4,
 1,
 4,
 1,
 7,
 1,
 2,
 5,
 3,
 2,
 2,
 4,
 4,
 2,
 4,
 6,
 7,
 1,
 2,
 4,
 5,
 2,
 4,
 4,
 7,
 2,
 2,
 6,
 5,
 1,
 2,
 4,
 4]